# 민원데이터 분석

민원데이터의 시각화분석을 통한 정책개선 아이디어 제시
- □ 분석방법

  ㅇ 국민권익위원회에서 제공하는 데이터를 활용하여 대한민국 국민의 민원 트렌드 분석 및 정책 개선에 기여할 수 있는 시각화 분석 
  
  ㅇ 대회에서 제공하는 데이터 外 민원 데이터와 융합하여 분석의 완성도를 높일 수 있는 개인/팀 별 수집 데이터* 활용 가능 
     * SNS, 신문기사 및 각종 공개데이터 및 공공데이터 등 

ㅇ 결과제출처: do231@korea.kr (제출마감일 18시까지 도착분에 한해 유효)

## 사용가능 OPEN API 기능 목록
- 급증 키워드 정보
- 핵심 키워드 정보
- 민원분석 분류체계 정보
- 맞춤형 통계 정보
- 키워드 트렌드 정보
- 유사사례 정보
- 연관어 분석 정보
- 오늘의 민원 이슈
- 민원발생 기관 순위
- 민원발생 지역 순위
- 키워드 기반 민원 건수 정보
- 지역 인구수 대비 민원 현황 정보
- 최다 민원 키워드 정보

In [1]:
open_api_key = "cjE984BWrGFH3DFA2kLDoNx7j950nWjp6Ajz0CBaEABY5arYHY1C80RNBw6ptmayEV6B0zzpOfAbB73miZ4gnA%3D%3D"

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df = pd.read_excel('./civil_right_without_password.xlsx')
df.head()

,제목,질문내용,답변내용,등록일시,최상위기관명,분야코드,분야명,담당부서명
0,의성군 귀농귀촌 정책 문의,의성군 귀농귀촌 정책 문의,1. 우선 저희 의성군으로 귀농·귀촌하고자 하셔서 감사합니다.\n \n2. 귀농·귀...,2021-08-27 오후 3:31:44,경상북도 의성군,20010010,행정/자치/안전,농업기술센터
1,불법주정차 신고합니다,5대 불법주정차 과태료 부과 대상,1. 귀하의 가정에 평안함과 행복을 기원합니다. 귀하께서 국민신문고로 요청하신 사항...,2021-08-27 오후 1:49:33,경상북도 의성군,20010010,행정/자치/안전,일자리창출과
2,아동수당 지급 대상,아동수당 지급 대상이 궁금합니다.,1. 안녕하십니까? 귀하께서 문의하신 질의에 대한 답변을 다음과 같이 알려드립니다....,2021-08-27 오후 4:19:51,경상북도 의성군,20010010,행정/자치/안전,복지과
3,광견병 예방 접종 관련 문의,광견병 예방접종 질문사항.\n1. 의성군에서 광견병 접종 예방 계획이 있으신지요?...,안녕하십니까?\n1. 귀하께서 국민신문고를 통해 신청하신 민원에 대한 검토 결과를 ...,2021-08-27 오후 4:18:09,경상북도 의성군,20010010,행정/자치/안전,농축산과
4,아동수당 지급 금액과 지급 일자,아동수당 지급 금액과 지급 일자가 궁금합니다.,1. 안녕하십니까? 귀하께서 문의하신 질의에 대한 답변을 다음과 같이 알려드립니다....,2021-08-27 오후 4:24:24,경상북도 의성군,20010010,행정/자치/안전,복지과


In [22]:
len(df)

57249

In [9]:
df['분야명'].value_counts()

행정/자치/안전       18253
국토/교통/농림/해양    13117
교육/문화/체육/관광    11109
재정/금융/소비자       5003
산업/방송/통신/과학     2995
국방/보훈/외교/통일     2216
노동/환경           1772
법제/사법           1414
식품의약품            783
기타               587
Name: 분야명, dtype: int64

In [10]:
df['분야코드'].value_counts()

20010010    18253
20010050    13117
20010040    11109
20010030     5003
20010070     2995
20010060     2216
20010090     1772
20010020     1414
200100B0      783
200100A0      587
Name: 분야코드, dtype: int64

In [11]:
df['담당부서명'].value_counts()

납세자보호담당관    2509
운영지원과       2260
감사담당관       1791
행정법무담당관     1777
민원여권과       1722
            ... 
산림병해충과         1
수질관리과          1
성북세무서          1
경남서부세관         1
항공관제국          1
Name: 담당부서명, Length: 1404, dtype: int64

In [26]:
df['분야명-담당부서명'] = df['분야명']+'-'+df['담당부서명']
df['분야명-담당부서명'].value_counts()[:60]

재정/금융/소비자-납세자보호담당관       2509
산업/방송/통신/과학-행정법무담당관      1777
행정/자치/안전-감사담당관           1753
행정/자치/안전-민원여권과           1722
국토/교통/농림/해양-운영지원과        1349
교육/문화/체육/관광-총무과          1027
행정/자치/안전-청문감사인권관          923
재정/금융/소비자-규제개혁법무담당관       914
국토/교통/농림/해양-선원해사안전과       882
교육/문화/체육/관광-행정지원과         825
국토/교통/농림/해양-항만물류과         818
법제/사법-혁신행정담당관             793
식품의약품-고객지원담당관             782
교육/문화/체육/관광-행정과           739
국토/교통/농림/해양-해양수산환경과       716
국토/교통/농림/해양-고객지원담당관       712
행정/자치/안전-법무감사혁신담당관        676
노동/환경-운영지원과               675
산업/방송/통신/과학-규제개혁법무담당관     625
행정/자치/안전-시민봉사담당관          619
국토/교통/농림/해양-도로안전운영과       614
행정/자치/안전-민원봉사과            597
국토/교통/농림/해양-시설안전관리과       513
국토/교통/농림/해양-항로표지과         434
교육/문화/체육/관광-중등교육지원과       423
행정/자치/안전-경무과              407
행정/자치/안전-혁신소통기획관          390
행정/자치/안전-민원토지과            380
행정/자치/안전-민원지적과            375
교육/문화/체육/관광-학교운영지원과       349
행정/자치/안전-통합민원과            347
교육/문화/체육/관광-대학재정장학과       301
교육/문화/체육/관광-학교지원과         297
교육/문화/체육/관

In [3]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' 
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 

def clean_punc(text, punct, mapping): 
    for p in mapping: 
        text = text.replace(p, mapping[p]) 
        
    for p in punct: 
        text = text.replace(p, f' {p} ') 
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''} 
    for s in specials: 
        text = text.replace(s, specials[s]) 
        
    return text.strip()

def clean_text(texts): 
    corpus = [] 
    for i in range(0, len(texts)): 
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation 
        review = re.sub(r'\d+','', str(texts[i]))# remove number 
        review = review.lower() #lower case 
        review = re.sub(r'\s+', ' ', review) #remove extra space 
        review = re.sub(r'<[^>]+>','',review) #remove Html tags 
        review = re.sub(r'\s+', ' ', review) #remove spaces 
        review = re.sub(r"^\s+", '', review) #remove space from start 
        review = re.sub(r'\s+$', '', review) #remove space from the end 
        
        corpus.append(review) 
        
        return corpus



https://ebbnflow.tistory.com/246

https://wikidocs.net/92961

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\good5\appdata\local\temp\pip-req-build-juefg2yw
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268598 sha256=a6cf813f5246d17dfa90fb08301b873345dded2f1d2583f2f64eece2d3666872
  Stored in directory: C:\Users\good5\AppData\Local\Temp\pip-ephem-wheel-cache-tid96_bu\wheels\79\a0\33\16f2cd03d21f76a663f5d69a0b96f0351335385349136fbd03
Successfully built pykospacing


In [6]:
from pykospacing import spacing

ImportError: cannot import name 'spacing' from 'pykospacing' (C:\Users\good5\Anaconda3\lib\site-packages\pykospacing\__init__.py)

In [ ]:
print(spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다."))

In [ ]:
!pip install kss

In [ ]:
import kss 

s = "회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요 다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다 강남역 맛집 토끼정의 외부 모습." 
for sent in kss.split_sentences(s): 
    print(sent)

In [ ]:
pip install py-hanspell

In [ ]:
from hanspell import spell_checker

result = spell_checker.check(u'안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.') 
result.as_dict() # dict로 출력
